In [ ]:
%pip install torch

In [ ]:
%pip install transformers

In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import pandas as pd
import numpy as np
from datetime import datetime
import re


In [2]:
tweet = pd.read_csv("stock_tweets.csv")

In [3]:
tweet_tesla = tweet[tweet["Company Name"] ==  'Tesla, Inc.']
tweet_tesla

,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."
...,...,...,...,...
37417,2021-09-30 02:52:38+00:00,Playing in the dirt and #chasingsunsets\n@tesl...,TSLA,"Tesla, Inc."
37418,2021-09-30 02:40:26+00:00,I agree with @freshjiva that $TSLA ‘s EV busin...,TSLA,"Tesla, Inc."
37419,2021-09-30 01:59:02+00:00,Hold. On. Tight. $TSLA,TSLA,"Tesla, Inc."
37420,2021-09-30 01:38:26+00:00,Get ready for a $TSLA _ _ _ _ _ _ Q3 delivery...,TSLA,"Tesla, Inc."


In [6]:
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # URL'leri kaldır
    tweet = re.sub(r'@\S+', '', tweet)  # Kullanıcı adlarını kaldır
    tweet = re.sub(r'#\S+', '', tweet)  # Hashtag'leri kaldır
    tweet = re.sub(r'RT\s+', '', tweet)  # RT etiketlerini kaldır
    "tweet = re.sub(r'[0-9]+', '', tweet)  # Sayıları kaldır"
    tweet = re.sub(r'[^\w\s]', '', tweet) # Noktalama işaretlerini ve özel karakterleri kaldır
    return tweet.strip()

In [7]:
# tweet'lerini filtreleyin ve temizleyin
tesla_tweets = tweet[tweet["Company Name"] ==  'Tesla, Inc.']['Tweet']
tweet_tesla['Cleaned_Tweet']  = tesla_tweets.apply(clean_tweet)

/var/folders/qz/0bd550q55tvf84nvlfh_vk6w0000gn/T/ipykernel_23376/3608169329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_tesla['Cleaned_Tweet']  = tesla_tweets.apply(clean_tweet)


In [8]:
tweet_tesla.sample(5)

,Date,Tweet,Stock Name,Company Name,Cleaned_Tweet
7602,2022-06-30 14:17:37+00:00,Luxury goods like $TSLA are the first to get c...,TSLA,"Tesla, Inc.",Luxury goods like TSLA are the first to get cr...
23292,2022-01-25 23:21:26+00:00,@TomHarnisch @Tesla legal told me not to reply...,TSLA,"Tesla, Inc.",legal told me not to reply to this
16411,2022-04-06 18:33:35+00:00,@GailAlfarATX @JoeTegtmeyer @Tesla @elonmusk @...,TSLA,"Tesla, Inc.",I so hard want to be at the Cyber Rodeo I hate...
32306,2021-11-07 23:39:02+00:00,NEWS: Wedbush analyst Daniel Ives reiterated a...,TSLA,"Tesla, Inc.",NEWS Wedbush analyst Daniel Ives reiterated an...
5259,2022-07-28 18:59:02+00:00,@TibaStefan1 @Tesla @elonmusk @gjurvetson Yup....,TSLA,"Tesla, Inc.",Yup\n\nSome random on Y With the 3 I shifted...


In [9]:
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [10]:
tweet_counter = 0

def predict_sentiment(tweet, threshold=0.3):
    tokens = tokenizer.encode(tweet, return_tensors='pt')
    with torch.no_grad():
        outputs = model(tokens)
        probabilities = softmax(outputs.logits, dim=1).squeeze()

    max_prob, predicted_class = torch.max(probabilities, dim=0)
    global tweet_counter
    tweet_counter += 1
    print(f"{tweet_counter}")

    if max_prob > threshold:
        if predicted_class == 0:
            return "Very Negative"
        elif predicted_class == 1:
            return "Negative"
        elif predicted_class == 2:
            return "Neutral"
        elif predicted_class == 3:
            return "Positive"
        elif predicted_class == 4:
            return "Very Positive"
    return "Neutral"



In [11]:
pd.set_option('display.max_columns', None)

In [14]:
tweet_tesla['BERT_sentiment_label'] = tweet_tesla["Cleaned_Tweet"].apply(predict_sentiment)

26925
26926
26927
26928
26929
26930
26931
26932
26933
26934
26935
26936
26937
26938
26939
26940
26941
26942
26943
26944
26945
26946
26947
26948
26949
26950
26951
26952
26953
26954
26955
26956
26957
26958
26959
26960
26961
26962
26963
26964
26965
26966
26967
26968
26969
26970
26971
26972
26973
26974
26975
26976
26977
26978
26979
26980
26981
26982
26983
26984
26985
26986
26987
26988
26989
26990
26991
26992
26993
26994
26995
26996
26997
26998
26999
27000
27001
27002
27003
27004
27005
27006
27007
27008
27009
27010
27011
27012
27013
27014
27015
27016
27017
27018
27019
27020
27021
27022
27023
27024
27025
27026
27027
27028
27029
27030
27031
27032
27033
27034
27035
27036
27037
27038
27039
27040
27041
27042
27043
27044
27045
27046
27047
27048
27049
27050
27051
27052
27053
27054
27055
27056
27057
27058
27059
27060
27061
27062
27063
27064
27065
27066
27067
27068
27069
27070
27071
27072
27073
27074
27075
27076
27077
27078
27079
27080
27081
27082
27083
27084
27085
27086
27087
27088
27089
27090
2709

28295
28296
28297
28298
28299
28300
28301
28302
28303
28304
28305
28306
28307
28308
28309
28310
28311
28312
28313
28314
28315
28316
28317
28318
28319
28320
28321
28322
28323
28324
28325
28326
28327
28328
28329
28330
28331
28332
28333
28334
28335
28336
28337
28338
28339
28340
28341
28342
28343
28344
28345
28346
28347
28348
28349
28350
28351
28352
28353
28354
28355
28356
28357
28358
28359
28360
28361
28362
28363
28364
28365
28366
28367
28368
28369
28370
28371
28372
28373
28374
28375
28376
28377
28378
28379
28380
28381
28382
28383
28384
28385
28386
28387
28388
28389
28390
28391
28392
28393
28394
28395
28396
28397
28398
28399
28400
28401
28402
28403
28404
28405
28406
28407
28408
28409
28410
28411
28412
28413
28414
28415
28416
28417
28418
28419
28420
28421
28422
28423
28424
28425
28426
28427
28428
28429
28430
28431
28432
28433
28434
28435
28436
28437
28438
28439
28440
28441
28442
28443
28444
28445
28446
28447
28448
28449
28450
28451
28452
28453
28454
28455
28456
28457
28458
28459
28460
2846

29664
29665
29666
29667
29668
29669
29670
29671
29672
29673
29674
29675
29676
29677
29678
29679
29680
29681
29682
29683
29684
29685
29686
29687
29688
29689
29690
29691
29692
29693
29694
29695
29696
29697
29698
29699
29700
29701
29702
29703
29704
29705
29706
29707
29708
29709
29710
29711
29712
29713
29714
29715
29716
29717
29718
29719
29720
29721
29722
29723
29724
29725
29726
29727
29728
29729
29730
29731
29732
29733
29734
29735
29736
29737
29738
29739
29740
29741
29742
29743
29744
29745
29746
29747
29748
29749
29750
29751
29752
29753
29754
29755
29756
29757
29758
29759
29760
29761
29762
29763
29764
29765
29766
29767
29768
29769
29770
29771
29772
29773
29774
29775
29776
29777
29778
29779
29780
29781
29782
29783
29784
29785
29786
29787
29788
29789
29790
29791
29792
29793
29794
29795
29796
29797
29798
29799
29800
29801
29802
29803
29804
29805
29806
29807
29808
29809
29810
29811
29812
29813
29814
29815
29816
29817
29818
29819
29820
29821
29822
29823
29824
29825
29826
29827
29828
29829
2983

31032
31033
31034
31035
31036
31037
31038
31039
31040
31041
31042
31043
31044
31045
31046
31047
31048
31049
31050
31051
31052
31053
31054
31055
31056
31057
31058
31059
31060
31061
31062
31063
31064
31065
31066
31067
31068
31069
31070
31071
31072
31073
31074
31075
31076
31077
31078
31079
31080
31081
31082
31083
31084
31085
31086
31087
31088
31089
31090
31091
31092
31093
31094
31095
31096
31097
31098
31099
31100
31101
31102
31103
31104
31105
31106
31107
31108
31109
31110
31111
31112
31113
31114
31115
31116
31117
31118
31119
31120
31121
31122
31123
31124
31125
31126
31127
31128
31129
31130
31131
31132
31133
31134
31135
31136
31137
31138
31139
31140
31141
31142
31143
31144
31145
31146
31147
31148
31149
31150
31151
31152
31153
31154
31155
31156
31157
31158
31159
31160
31161
31162
31163
31164
31165
31166
31167
31168
31169
31170
31171
31172
31173
31174
31175
31176
31177
31178
31179
31180
31181
31182
31183
31184
31185
31186
31187
31188
31189
31190
31191
31192
31193
31194
31195
31196
31197
3119

32401
32402
32403
32404
32405
32406
32407
32408
32409
32410
32411
32412
32413
32414
32415
32416
32417
32418
32419
32420
32421
32422
32423
32424
32425
32426
32427
32428
32429
32430
32431
32432
32433
32434
32435
32436
32437
32438
32439
32440
32441
32442
32443
32444
32445
32446
32447
32448
32449
32450
32451
32452
32453
32454
32455
32456
32457
32458
32459
32460
32461
32462
32463
32464
32465
32466
32467
32468
32469
32470
32471
32472
32473
32474
32475
32476
32477
32478
32479
32480
32481
32482
32483
32484
32485
32486
32487
32488
32489
32490
32491
32492
32493
32494
32495
32496
32497
32498
32499
32500
32501
32502
32503
32504
32505
32506
32507
32508
32509
32510
32511
32512
32513
32514
32515
32516
32517
32518
32519
32520
32521
32522
32523
32524
32525
32526
32527
32528
32529
32530
32531
32532
32533
32534
32535
32536
32537
32538
32539
32540
32541
32542
32543
32544
32545
32546
32547
32548
32549
32550
32551
32552
32553
32554
32555
32556
32557
32558
32559
32560
32561
32562
32563
32564
32565
32566
3256

...

59761
59762
59763
59764
59765
59766
59767
59768
59769
59770
59771
59772
59773
59774
59775
59776
59777
59778
59779
59780
59781
59782
59783
59784
59785
59786
59787
59788
59789
59790
59791
59792
59793
59794
59795
59796
59797
59798
59799
59800
59801
59802
59803
59804
59805
59806
59807
59808
59809
59810
59811
59812
59813
59814
59815
59816
59817
59818
59819
59820
59821
59822
59823
59824
59825
59826
59827
59828
59829
59830
59831
59832
59833
59834
59835
59836
59837
59838
59839
59840
59841
59842
59843
59844
59845
59846
59847
59848
59849
59850
59851
59852
59853
59854
59855
59856
59857
59858
59859
59860
59861
59862
59863
59864
59865
59866
59867
59868
59869
59870
59871
59872
59873
59874
59875
59876
59877
59878
59879
59880
59881
59882
59883
59884
59885
59886
59887
59888
59889
59890
59891
59892
59893
59894
59895
59896
59897
59898
59899
59900
59901
59902
59903
59904
59905
59906
59907
59908
59909
59910
59911
59912
59913
59914
59915
59916
59917
59918
59919
59920
59921
59922
59923
59924
59925
59926
5992

61127
61128
61129
61130
61131
61132
61133
61134
61135
61136
61137
61138
61139
61140
61141
61142
61143
61144
61145
61146
61147
61148
61149
61150
61151
61152
61153
61154
61155
61156
61157
61158
61159
61160
61161
61162
61163
61164
61165
61166
61167
61168
61169
61170
61171
61172
61173
61174
61175
61176
61177
61178
61179
61180
61181
61182
61183
61184
61185
61186
61187
61188
61189
61190
61191
61192
61193
61194
61195
61196
61197
61198
61199
61200
61201
61202
61203
61204
61205
61206
61207
61208
61209
61210
61211
61212
61213
61214
61215
61216
61217
61218
61219
61220
61221
61222
61223
61224
61225
61226
61227
61228
61229
61230
61231
61232
61233
61234
61235
61236
61237
61238
61239
61240
61241
61242
61243
61244
61245
61246
61247
61248
61249
61250
61251
61252
61253
61254
61255
61256
61257
61258
61259
61260
61261
61262
61263
61264
61265
61266
61267
61268
61269
61270
61271
61272
61273
61274
61275
61276
61277
61278
61279
61280
61281
61282
61283
61284
61285
61286
61287
61288
61289
61290
61291
61292
6129

62494
62495
62496
62497
62498
62499
62500
62501
62502
62503
62504
62505
62506
62507
62508
62509
62510
62511
62512
62513
62514
62515
62516
62517
62518
62519
62520
62521
62522
62523
62524
62525
62526
62527
62528
62529
62530
62531
62532
62533
62534
62535
62536
62537
62538
62539
62540
62541
62542
62543
62544
62545
62546
62547
62548
62549
62550
62551
62552
62553
62554
62555
62556
62557
62558
62559
62560
62561
62562
62563
62564
62565
62566
62567
62568
62569
62570
62571
62572
62573
62574
62575
62576
62577
62578
62579
62580
62581
62582
62583
62584
62585
62586
62587
62588
62589
62590
62591
62592
62593
62594
62595
62596
62597
62598
62599
62600
62601
62602
62603
62604
62605
62606
62607
62608
62609
62610
62611
62612
62613
62614
62615
62616
62617
62618
62619
62620
62621
62622
62623
62624
62625
62626
62627
62628
62629
62630
62631
62632
62633
62634
62635
62636
62637
62638
62639
62640
62641
62642
62643
62644
62645
62646
62647
62648
62649
62650
62651
62652
62653
62654
62655
62656
62657
62658
62659
6266

63865
63866
63867
63868
63869
63870
63871
63872
63873
63874
63875
63876
63877
63878
63879
63880
63881
63882
63883
63884
63885
63886
63887
63888
63889
63890
63891
63892
63893
63894
63895
63896
63897
63898
63899
63900
63901
63902
63903
63904
63905
63906
63907
63908
63909
63910
63911
63912
63913
63914
63915
63916
63917
63918
63919
63920
63921
63922
63923
63924
63925
63926
63927
63928
63929
63930
63931
63932
63933
63934
63935
63936
63937
63938
63939
63940
63941
63942
63943
63944
63945
63946
63947
63948
63949
63950
63951
63952
63953
63954
63955
63956
63957
63958
63959
63960
63961
63962
63963
63964
63965
63966
63967
63968
63969
63970
63971
63972
63973
63974
63975
63976
63977
63978
63979
63980
63981
63982
63983
63984
63985
63986
63987
63988
63989
63990
63991
63992
63993
63994
63995
63996
63997
63998
63999
64000
64001
64002
64003
64004
64005
64006
64007
64008
64009
64010
64011
64012
64013
64014
64015
64016
64017
64018
64019
64020
64021
64022
64023
64024
64025
64026
64027
64028
64029
64030
6403

/var/folders/qz/0bd550q55tvf84nvlfh_vk6w0000gn/T/ipykernel_23376/1110349313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_tesla['BERT_sentiment_label'] = tweet_tesla["Cleaned_Tweet"].apply(predict_sentiment)


In [15]:
tweet_tesla.to_csv("BERT_falanfilan.csv",index=False)

In [19]:
tweet_tesla.sample(8)

,Date,Tweet,Stock Name,Company Name,Cleaned_Tweet,BERT_sentiment_label
26874,2021-12-28 16:50:50+00:00,$TSLA price target raised from $1100 to $1400 ...,TSLA,"Tesla, Inc.",TSLA price target raised from 1100 to 1400 at ...,Neutral
23500,2022-01-24 15:49:29+00:00,"Do not follow, for learning.\n\nI’ve converted...",TSLA,"Tesla, Inc.",Do not follow for learning\n\nIve converted so...,Very Negative
10076,2022-05-29 19:02:06+00:00,And boom. We are finally an all electric ⚡️ fa...,TSLA,"Tesla, Inc.",And boom We are finally an all electric famil...,Very Positive
17237,2022-03-30 02:13:27+00:00,That blue hits different when the sun sets.👌🏾@...,TSLA,"Tesla, Inc.",That blue hits different when the sun sets,Negative
27439,2021-12-22 20:42:13+00:00,Looking good. Tesla holding 3 Gordons. Get you...,TSLA,"Tesla, Inc.",Looking good Tesla holding 3 Gordons Get your ...,Very Positive
13904,2022-04-26 14:30:32+00:00,@shaunking @elonmusk @Tesla I knew it was too ...,TSLA,"Tesla, Inc.",I knew it was too good to be true You werent g...,Very Negative
18224,2022-03-22 11:01:44+00:00,"""@Tesla's market share is still collapsing. We...",TSLA,"Tesla, Inc.",market share is still collapsing We actually t...,Negative
23727,2022-01-21 23:25:11+00:00,$AAPL $MSFT $GOOGL $AMZN $TSLA https://t.co/dp...,TSLA,"Tesla, Inc.",AAPL MSFT GOOGL AMZN TSLA,Neutral


In [16]:
result_df = pd.merge(tweet_tesla, dffff, left_index=True, right_index=True, how='left', suffixes=('_df1', '_df2'))
result_df['BERT_sentiment_label_df1'].fillna(result_df['BERT_sentiment_label_v2'], inplace=True)
result_df.drop('Tweet_df2', axis=1, inplace=True)

NameError: name 'dffff' is not defined

In [17]:
tweet_tesla.replace("None",np.nan,inplace=True)

/var/folders/qz/0bd550q55tvf84nvlfh_vk6w0000gn/T/ipykernel_23376/2450281771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_tesla.replace("None",np.nan,inplace=True)


In [18]:
result_df['BERT_sentiment_label_df1'].fillna(result_df['BERT_sentiment_label_v2'], inplace=True)



NameError: name 'result_df' is not defined

In [ ]:
result_df["BERT_sentiment_label_df1"].unique()

In [ ]:
result_df.columns

In [ ]:
result_df.rename(columns={"Date_df1" : "Date","Tweet_df1" : "Tweet","Stock Name_df1":"Stock Name","Company Name_df1":"Company Name","BERT_sentiment_label_df1":"BERT_sentiment_label"},inplace=True)

In [2]:
result_df = pd.read_csv("BERT_falanfilan.csv")

In [9]:
result_df["Sentiment_Point"] = None

In [3]:
sentiment_mapping = {"Very Negative" : -2,
                    "Negative" : -1,
                    "Neutral" : 0,
                    "Positive" : 1,
                    "Very Positive" : 2}

In [11]:
for index,row in result_df.iterrows():
    result_df.iloc[index]["Sentiment_Point"] = sentiment_mapping[row["BERT_sentiment_label"]]

In [12]:
result_df

,Date,Tweet,Stock Name,Company Name,BERT_sentiment_label,Sentiment_Point
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",Very Positive,2
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",Very Negative,-2
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",Very Negative,-2
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",Very Negative,-2
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",Very Negative,-2
...,...,...,...,...,...,...
37417,2021-09-30 02:52:38+00:00,Playing in the dirt and #chasingsunsets\n@tesl...,TSLA,"Tesla, Inc.",Neutral,0
37418,2021-09-30 02:40:26+00:00,I agree with @freshjiva that $TSLA ‘s EV busin...,TSLA,"Tesla, Inc.",Very Negative,-2
37419,2021-09-30 01:59:02+00:00,Hold. On. Tight. $TSLA,TSLA,"Tesla, Inc.",Neutral,0
37420,2021-09-30 01:38:26+00:00,Get ready for a $TSLA _ _ _ _ _ _ Q3 delivery...,TSLA,"Tesla, Inc.",Very Positive,2


In [13]:
result_df["Date"] = pd.to_datetime(result_df["Date"]).dt.date

In [14]:
result_result_df = result_df.groupby('Date')['Sentiment_Point'].sum().reset_index()


In [17]:
def process_sentiment(df):
    # Tarih sütununu tarih türüne dönüştürmek
    df['Date'] = pd.to_datetime(df['Date'])

    # Puanları kategorize eden fonksiyonlar
    def positive(x): return x > 0
    def negative(x): return x < 0
    def neutral(x): return x == 0

    # Günlük bazda puanları saymak
    grouped = df.groupby('Date')['Sentiment_Point'].agg(
        positive_count=lambda x: x.apply(positive).sum(),
        negative_count=lambda x: x.apply(negative).sum(),
        neutral_count=lambda x: x.apply(neutral).sum()
    )

    return grouped.reset_index()

# Yeni DataFrame'i elde etmek
new_df = process_sentiment(result_df)

    

In [23]:
result_result_df["Date"] = pd.to_datetime(result_result_df["Date"])

In [24]:
new_new_df = pd.merge(result_result_df,new_df,how='left',on=["Date"])

In [27]:
new_new_df["total_tweet_count"] = new_new_df["positive_count"] + new_new_df["negative_count"] + new_new_df["neutral_count"]

In [29]:
new_new_df.sum()

/var/folders/0t/rpp9ntlx6xl16gfchdbbwjd83b2ts6/T/ipykernel_4133/2762608870.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  new_new_df.sum()


Sentiment_Point      -21776
positive_count         9790
negative_count        21084
neutral_count          6548
total_tweet_count     37422
dtype: object

In [30]:
stock_finance_data = pd.read_csv("stock_yfinance_data.csv")


In [31]:
tesla_stock_finance =stock_finance_data[stock_finance_data["Stock Name"] == "TSLA"]


In [32]:
tesla_stock_finance["Close_Diff"] = tesla_stock_finance["Close"].pct_change()


/var/folders/0t/rpp9ntlx6xl16gfchdbbwjd83b2ts6/T/ipykernel_4133/3860090083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_stock_finance["Close_Diff"] = tesla_stock_finance["Close"].pct_change()


In [42]:
tesla_stock_finance.drop(["Open","High","Low","Adj Close","Volume"], axis = 1,inplace=True)

/var/folders/0t/rpp9ntlx6xl16gfchdbbwjd83b2ts6/T/ipykernel_4133/1595488923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_stock_finance.drop(["Open","High","Low","Adj Close","Volume"], axis = 1,inplace=True)


In [44]:
tesla_stock_finance["Date"] = pd.to_datetime(tesla_stock_finance["Date"]) 

/var/folders/0t/rpp9ntlx6xl16gfchdbbwjd83b2ts6/T/ipykernel_4133/2741629571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_stock_finance["Date"] = pd.to_datetime(tesla_stock_finance["Date"])


In [45]:
df_df = pd.merge(new_new_df,tesla_stock_finance,how="left",on=["Date"])

In [54]:
tesla_stock_finance.dropna(inplace=True)

/var/folders/0t/rpp9ntlx6xl16gfchdbbwjd83b2ts6/T/ipykernel_4133/4081354181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_stock_finance.dropna(inplace=True)


In [64]:
def merge_dataframes(df1, df2):
    merged_df = pd.merge(df1, df2, on='Date', how='inner')
    return merged_df

# 1. Hafta Sonu Verilerini Silerek Korelasyon Hesaplama
new_new_df_weekdays = new_new_df[new_new_df['Date'].dt.dayofweek < 5]
merged_df_weekdays = merge_dataframes(new_new_df_weekdays, tesla_stock_finance)
merged_df_weekdays = merged_df_weekdays.dropna(subset=['Sentiment_Point', 'Close_Diff'])
merged_df_weekdays = merged_df_weekdays.replace([np.inf, -np.inf], np.nan).dropna(subset=['Sentiment_Point', 'Close_Diff'])


correlation_weekdays = merged_df_weekdays['Sentiment_Point'].corr(merged_df_weekdays['Close_Diff'])

# 2. Hafta Sonu Verilerini Cuma Gününe Yuvarlayarak Korelasyon Hesaplama
new_new_df_friday = new_new_df.copy()
new_new_df_friday['Date'] = new_new_df_friday['Date'].apply(lambda x: x - pd.Timedelta(days=x.dayofweek - 4) if x.dayofweek > 4 else x)
merged_df_friday = merge_dataframes(new_new_df_friday, tesla_stock_finance)
merged_df_friday = merged_df_friday.dropna(subset=['Sentiment_Point', 'Close_Diff'])
merged_df_friday = merged_df_friday.replace([np.inf, -np.inf], np.nan).dropna(subset=['Sentiment_Point', 'Close_Diff'])

correlation_friday = merged_df_friday['Sentiment_Point'].corr(merged_df_friday['Close_Diff'])


# 3. Hafta Sonu Verilerini Pazartesi Gününe Yuvarlayarak Korelasyon Hesaplama
new_new_df_monday = new_new_df.copy()
new_new_df_monday['Date'] = new_new_df_monday['Date'].apply(lambda x: x + pd.Timedelta(days=7 - x.dayofweek) if x.dayofweek > 4 else x)
merged_df_monday = merge_dataframes(new_new_df_monday, tesla_stock_finance)
merged_df_monday = merged_df_monday.dropna(subset=['Sentiment_Point', 'Close_Diff'])
merged_df_monday = merged_df_monday.replace([np.inf, -np.inf], np.nan).dropna(subset=['Sentiment_Point', 'Close_Diff'])

correlation_monday = merged_df_monday['Sentiment_Point'].corr(merged_df_monday['Close_Diff'])

# Korelasyon değerlerini yazdırma
print("Hafta içi verileriyle korelasyon:", correlation_weekdays)
print("Cuma gününe yuvarlanmış verilerle korelasyon:", correlation_friday)
print("Pazartesi gününe yuvarlanmış verilerle korelasyon:", correlation_monday)

Hafta içi verileriyle korelasyon: 0.06709304570959403
Cuma gününe yuvarlanmış verilerle korelasyon: 0.04272184807926748
Pazartesi gününe yuvarlanmış verilerle korelasyon: 0.08342159794576198


In [ ]:
#BİR ÜSTTEKİ CELL'DE KALDIM
#YETER KÂFİ
#SAAT OLMUS 1 AQ YARIN CUMA YA MESAİ VAR 9DA LANET ETTİM YETERRRRRRRRRRRRRRRRRRRR

In [ ]:
tweet_data = result_result_df  # Örnek isim, dosya adınıza göre değiştirin

# İkinci veri setini yükleyin (hisse senedi fiyatları)
stock_data = tesla_stock_finance  # Örnek isim, dosya adınıza göre değiştirin

# Tarih sütunlarını datetime tipine çevirin
tweet_data['Date'] = pd.to_datetime(tweet_data['Date'])
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Günlük bazda birleştirme işlemi
merged_data = pd.merge(tweet_data, stock_data, how='inner', on='Date')

# Tüm tarihler için korelasyonları depolamak için bir liste oluşturun
correlation_results = []

# Tüm tarihler üzerinde dönerek günlük korelasyon analizi yapın
for date in merged_data['Date'].unique():
    daily_data = merged_data[merged_data['Date'] == date]
    correlation_matrix = daily_data[['Sentiment_Point', 'Close']].corr()
    correlation_results.append({'Date': date, 'Correlation': correlation_matrix.loc['Sentiment_Point', 'Close']})

# Sonuçları bir DataFrame'e çevirin
correlation_df = pd.DataFrame(correlation_results)

# Sonuçları yazdırın
print(correlation_df)

In [ ]:
tweet

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')

In [ ]:
for index, row in tweet.iterrows():
    tweet = row['body']  # 'body' is just an example, replace it with the actual column name
    inputs = tokenizer(tweet, return_tensors="pt")
    # Rest of your code for tweet analysis here

    outputs = model(inputs)

    predictions = outputs.logits
    predicted_class = torch.argmax(predictions).item()

    class_labels = ['Negative', 'Neutral', 'Positive']
    sentiment = class_labels[predicted_class]

    print(f"Tweet: {tweet}, Sentiment: {sentiment}")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch

# Load your dataset
# Assuming you have a DataFrame 'tweet_trainthing' with columns 'body' and 'BERT_sentiment_label'
df = tweet_trainthing

# Map sentiment labels to numerical values
sentiment_mapping = {
    'Very Negative': 0,
    'Negative': 1,
    'Neutral': 2,
    'Positive': 3,
    'Very Positive': 4
}

# Replace NaN labels with a default label (e.g., -1)
df['BERT_sentiment_label'] = df['BERT_sentiment_label'].map(sentiment_mapping)
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2,train_size=0.8, random_state=42)

# ... (Rest of the code remains unchanged)


# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = int(self.labels.iloc[idx])

        # Tokenize the text
        tokens = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            max_length=128,  # Adjust as needed
            return_tensors='pt'
        )

        return {
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Initialize the tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=5)  # 5 sentiment classes

# Create instances of the custom dataset
train_dataset = CustomDataset(texts=train_df['body'], labels=train_df['BERT_sentiment_label'], tokenizer=tokenizer)
val_dataset = CustomDataset(texts=val_df['body'], labels=val_df['BERT_sentiment_label'], tokenizer=tokenizer)

# Create DataLoader instances
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Define the training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 3  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Validation
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)

            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = total_correct / total_samples
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy:.4f}')

# Save the fine-tuned model
model.save_pretrained('fine_tuned_xlnet_sentiment')


In [ ]:
from transformers import XLNetForSequenceClassification, XLNetTokenizer
import torch
import torch.nn.functional as F

# Step 1: Load the Pretrained Model
model_path = 'fine_tuned_xlnet_model'
model = XLNetForSequenceClassification.from_pretrained(model_path)

# Step 2: Tokenization
tokenizer = XLNetTokenizer.from_pretrained(model_path)
text = "Your input text goes here."
tokens = tokenizer.encode(text, return_tensors='pt')

# Step 3: Inference
with torch.no_grad():
    outputs = model(tokens)

# Step 4: Post-process Output
logits = outputs.logits
probabilities = F.softmax(logits, dim=-1)

# Step 5: Interpret Results
predicted_class = torch.argmax(probabilities, dim=-1).item()

# Print the results
print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")

In [ ]:
tweet.to_csv("sentiment2k.csv")

indikatör

In [ ]:
df = 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# Örnek verilerin oluşturulması
np.random.seed(0)
dates = pd.date_range('20230101', periods=60)
data = pd.DataFrame(index=dates)
data['positive_score'] = np.random.rand(60)  # Pozitif duygu skorları
data['negative_score'] = np.random.rand(60)  # Negatif duygu skorları
data['price'] = np.random.rand(60) * 100  # Örnek hisse senedi fiyatları

In [ ]:
# Basit Hareketli Ortalama İndikatörü
data['moving_average'] = data['price'].rolling(window=5).mean()

# Ertesi günün fiyatını tahmin etmek için sütunları kaydırma
data['next_day_price'] = data['price'].shift(-1)

# NaN değerlerin temizlenmesi
data = data.dropna()

In [ ]:
# Özellikler ve hedef
features = data[['positive_score', 'negative_score', 'moving_average']]
target = data['next_day_price']

In [ ]:
# Veri setinin eğitim ve test setlerine bölünmesi
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

In [ ]:
# Modelin oluşturulması ve eğitimi
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Tahmin ve değerlendirme
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f"Test MSE: {mse}")

In [ ]:

# Tahminlerin gösterilmesi
predicted_data = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(predicted_data)
